In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook
import csv
import time
import os
import requests

In [2]:
# ==============================
# READ EXCEL
# ==============================
import re

wb = load_workbook("data.xlsx")
sheet = wb.active

barcodes = []
skipped_no_numbers = []
empty_cells = 0
row_count = 0

for row in sheet.iter_rows(min_row=3, values_only=True):
    row_count += 1
    code = row[0]
    
    if not code or not str(code).strip():
        empty_cells += 1
        continue
        
    # Extract only numeric portion from barcode
    numeric_code = re.sub(r'\D', '', str(code))
    if numeric_code:  # Only add if there are numbers
        barcodes.append(numeric_code)
    else:
        # Track barcodes with no numbers
        skipped_no_numbers.append(str(code))

print(f"Total rows processed: {row_count}")
print(f"Empty cells: {empty_cells}")
print(f"Entries with no numbers: {len(skipped_no_numbers)}")
print(f"Loaded barcodes: {len(barcodes)}")
print(f"\nFirst 5 barcodes: {barcodes[:5]}")

if skipped_no_numbers:
    print(f"\n⚠️ Skipped {len(skipped_no_numbers)} entries (no numbers found):")
    for s in skipped_no_numbers:
        print(f"  - '{s}'")

Total rows processed: 80
Empty cells: 0
Entries with no numbers: 1
Loaded barcodes: 79

First 5 barcodes: ['34356790304', '34116769951', '34116860912', '34216792228', '34116864057']

⚠️ Skipped 1 entries (no numbers found):
  - 'كليبر بلو خلفي شمال'


In [3]:
# ==============================
# DRIVER SETUP - OPTIMIZED
# ==============================
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

# ✅ block noisy stuff + disable images for faster scraping
prefs = {
    "profile.default_content_setting_values.notifications": 2,
    "profile.default_content_setting_values.geolocation": 2,
    "profile.default_content_setting_values.popups": 0,
    "profile.managed_default_content_settings.images": 2,  # Disable images
}

options.add_experimental_option("prefs", prefs)
options.add_argument("--blink-settings=imagesEnabled=false")  # Extra image blocking
options.add_argument("--disable-gpu")  # Reduce GPU usage
options.add_argument("--no-sandbox")  # Faster startup

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 10)  # Reduced from 15 to 10 seconds

cookie_handled = False
all_product_data = []

print("✅ Driver setup complete (optimized mode)")

✅ Driver setup complete (optimized mode)


In [4]:
# ==============================
# POPUP KILLER FUNCTIONS
# ==============================

def kill_overlays(driver):
    """Remove all modal popups, overlays, and enable scrolling"""
    driver.execute_script("""
        const selectors = [
            '.modal', '.popup', '.overlay', '.backdrop',
            '.cookie', '.cookies', '.consent',
            '[role="dialog"]',
            '[class*="modal"]',
            '[class*="popup"]',
            '[class*="overlay"]',
            '.ro-modal', '[data-ro]'
        ];

        selectors.forEach(sel => {
            document.querySelectorAll(sel).forEach(e => e.remove());
        });

        document.body.style.overflow = 'auto';
    """)


def handle_cookie():
    """Auto-accept cookies once"""
    global cookie_handled
    if cookie_handled:
        return

    try:
        cookie_btn = driver.find_element(By.XPATH, "//button[contains(., 'Accept') or contains(., 'Agree')]")
        cookie_btn.click()
        cookie_handled = True
    except:
        pass


def handle_subscription_popup():
    """Handle subscription popup - click 'Maybe Later' or close button (aggressive)"""
    popup_closed = False
    
    # Try multiple selectors for "Maybe Later" button
    try:
        maybe_later = driver.find_element(By.CSS_SELECTOR, 'button[data-ro="later"]')
        maybe_later.click()
        time.sleep(0.2)  # Brief pause after click
        popup_closed = True
    except:
        pass
    
    # Try alternative selector
    if not popup_closed:
        try:
            maybe_later = driver.find_element(By.CSS_SELECTOR, 'button.ro-btn.ro-secondary[data-ro="later"]')
            maybe_later.click()
            time.sleep(0.2)
            popup_closed = True
        except:
            pass
    
    # Try close button
    if not popup_closed:
        try:
            close_btn = driver.find_element(By.CSS_SELECTOR, 'button[data-ro="close"]')
            close_btn.click()
            time.sleep(0.2)
            popup_closed = True
        except:
            pass
    
    # Try alternative close button selector
    if not popup_closed:
        try:
            close_btn = driver.find_element(By.CSS_SELECTOR, 'button.ro-close[data-ro="close"]')
            close_btn.click()
            time.sleep(0.2)
            popup_closed = True
        except:
            pass
    
    return popup_closed


def close_extra_tabs(driver):
    """Close any surprise popup tabs"""
    main = driver.window_handles[0]
    for h in driver.window_handles:
        if h != main:
            driver.switch_to.window(h)
            driver.close()
    driver.switch_to.window(main)


def aggressive_popup_killer(driver):
    """Aggressively remove all popups without waiting"""
    try:
        # Kill all overlay elements
        kill_overlays(driver)
        # Try to handle subscription popup (non-blocking)
        handle_subscription_popup()
        # Handle cookies
        handle_cookie()
        # Close extra tabs
        close_extra_tabs(driver)
    except:
        pass


def safe_navigate(driver, url):
    """Navigate to URL and apply all protection layers"""
    driver.get(url)
    # Immediately kill any popups (don't wait)
    aggressive_popup_killer(driver)


print("✅ Popup killer functions loaded")

✅ Popup killer functions loaded


In [56]:
# ==============================
# TEST WITH FIRST BARCODE
# ==============================

from openpyxl import Workbook

test_barcode = barcodes[0]
print(f"Testing with first barcode: {test_barcode}")
print("=" * 60)

url = f"https://www.realoem.com/bmw/enUS/partxref?q={test_barcode}"

# Navigate to the page
safe_navigate(driver, url)

# Wait for the h1 element to appear with actual content (more specific than div.content)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.content h1")))

# Aggressively kill any popups that might have appeared
aggressive_popup_killer(driver)

print("✅ Page loaded and ready for scraping")

Testing with first barcode: 34356790304
✅ Page loaded and ready for scraping


In [41]:
# ==============================
# EXTRACT PART DATA - OPTIMIZED
# ==============================

# Check if part was not found (no extra delay needed)
part_found = True
try:
    error_div = driver.find_element(By.CSS_SELECTOR, "div.error.vs2")
    error_text = error_div.text.strip()
    
    if "not found" in error_text.lower():
        print(f"⚠️ Part not found: {error_text}")
        print("=" * 60)
        part_found = False
        
        # Store as not found data
        product_data = {
            "part_number": "NOT FOUND",
            "description": error_text,
            "vehicles": []
        }
except:
    # No error div found, continue with scraping
    pass

if part_found:
    # Wait for content to load
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.content h1")))
    
    # Extract part number and description (batch query for efficiency)
    part_number = driver.find_element(By.CSS_SELECTOR, "div.content h1").text
    description = driver.find_element(By.CSS_SELECTOR, "div.content h2").text
    
    print(f"Part Number: {part_number}")
    print(f"Description: {description}")
    
    # Extract details from dl (batch query)
    dl_data = {}
    try:
        dt_elements = driver.find_elements(By.CSS_SELECTOR, "div.content dl dt")
        dd_elements = driver.find_elements(By.CSS_SELECTOR, "div.content dl dd")
        
        for dt, dd in zip(dt_elements, dd_elements):
            key = dt.text.replace(":", "").strip()
            value = dd.text.strip() if dd.text.strip() else "-"
            dl_data[key] = value
            print(f"{key}: {value}")
    except Exception as e:
        print(f"Error extracting details: {e}")
    
    # Extract vehicle links
    vehicle_links = []
    try:
        links = driver.find_elements(By.CSS_SELECTOR, "div.partSearchResults ul li a")
        for link in links:
            vehicle_text = link.text
            vehicle_url = link.get_attribute("href")
            vehicle_links.append({"text": vehicle_text, "url": vehicle_url})
            print(f"Vehicle: {vehicle_text}")
    except Exception as e:
        print(f"Error extracting vehicle links: {e}")
    
    # Store the data
    product_data = {
        "part_number": part_number,
        "description": description,
        **dl_data,
        "vehicles": vehicle_links
    }

Part Number: 34356790304
Description: Brake pad wear sensor, rear
From: 09/01/2010
To: -
Weight: 0.057 kg
Price: -
Vehicle: X3 F25   (06/2009 — 08/2017)
Vehicle: X4 F26   (05/2013 — 03/2018)


In [43]:
# ==============================
# GO TO FIRST VEHICLE LINK
# ==============================

if vehicle_links:
    first_vehicle = vehicle_links[0]
    print(f"Navigating to first vehicle: {first_vehicle['text']}")
    print(f"URL: {first_vehicle['url']}")
    print("=" * 60)
    
    # Navigate to first vehicle link
    safe_navigate(driver, first_vehicle['url'])
    
    # Wait for the partSearchResults section to appear
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "partSearchResults")))
    
    # Aggressively kill any popups
    aggressive_popup_killer(driver)
    
    print("✅ Page loaded successfully")
else:
    print("⚠️ No vehicle links found")

Navigating to first vehicle: X3 F25   (06/2009 — 08/2017)
URL: https://www.realoem.com/bmw/enUS/partxref?q=34356790304&series=F25
✅ Page loaded successfully


In [44]:
# ==============================
# EXTRACT FIRST VEHICLE TAGS
# ==============================

try:
    # Find the partSearchResults section
    results_section = wait.until(
        EC.presence_of_element_located((By.CLASS_NAME, "partSearchResults"))
    )
    
    # Find the first <li> element in the list
    first_li = results_section.find_element(By.CSS_SELECTOR, "ul li:first-child")
    
    # Get the text content of the first <li>
    full_text = first_li.text.strip()
    
    # Extract text before the colon (vehicle tags)
    if ':' in full_text:
        vehicle_tags = full_text.split(':')[0].strip()
    else:
        vehicle_tags = full_text
    
    print("=" * 60)
    print("✅ First Vehicle Tags:")
    print(vehicle_tags)
    print("=" * 60)
    
except Exception as e:
    print(f"❌ Error extracting vehicle tags: {e}")

✅ First Vehicle Tags:
X3 F25, X3 18d, SUV, N47N, EUR, (WY11)


---
## FULL SCRAPING FOR ALL BARCODES
---

In [5]:
# ==============================
# SCRAPE ALL BARCODES - OPTIMIZED WITH TIMESTAMPS
# ==============================
from datetime import datetime

# Reset the data collection list
all_scraped_data = []

# Start timing
start_time = time.time()
start_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"🚀 Starting optimized scraping for {len(barcodes)} barcodes...")
print(f"   Start time: {start_datetime}")
print("=" * 60)

for idx, barcode in enumerate(barcodes, 1):
    item_start = time.time()
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"\n[{idx}/{len(barcodes)}] [{timestamp}] Processing: {barcode}", end=" ")
    
    try:
        # Step 1: Navigate to part search page
        url = f"https://www.realoem.com/bmw/enUS/partxref?q={barcode}"
        safe_navigate(driver, url)
        
        # Proactively kill popups before waiting
        time.sleep(0.5)  # Give page a moment to load
        aggressive_popup_killer(driver)
        
        # Step 2: Wait for either error message OR content (whichever appears first)
        try:
            WebDriverWait(driver, 10).until(
                lambda d: len(d.find_elements(By.CSS_SELECTOR, "div.error.vs2")) > 0 or 
                         len(d.find_elements(By.CSS_SELECTOR, "div.content h1")) > 0
            )
        except:
            # If timeout, aggressively kill popups and try again
            aggressive_popup_killer(driver)
            time.sleep(0.5)
            try:
                WebDriverWait(driver, 5).until(
                    lambda d: len(d.find_elements(By.CSS_SELECTOR, "div.error.vs2")) > 0 or 
                             len(d.find_elements(By.CSS_SELECTOR, "div.content h1")) > 0
                )
            except:
                # Still timing out, log and skip
                pass
        
        # Immediately kill any popups that appeared
        aggressive_popup_killer(driver)
        
        # Check if part was not found (check error FIRST before trying to extract data)
        part_not_found = False
        try:
            error_div = driver.find_element(By.CSS_SELECTOR, "div.error.vs2")
            error_text = error_div.text.strip()
            
            if "not found" in error_text.lower():
                item_time = time.time() - item_start
                print(f"⚠️ NOT FOUND ({item_time:.1f}s)")
                all_scraped_data.append({
                    "barcode": barcode,
                    "part_number": "NOT FOUND",
                    "description": error_text,
                    "from_date": "",
                    "to_date": "",
                    "weight": "",
                    "price": "",
                    "vehicle_count": 0,
                    "first_vehicle_tags": "",
                    "scrape_time_seconds": round(item_time, 2)
                })
                part_not_found = True
        except:
            # No error div found, continue with scraping
            pass
        
        # Skip to next barcode if not found
        if part_not_found:
            continue
        
        # Step 3: Extract part data - verify content exists first
        try:
            part_number = driver.find_element(By.CSS_SELECTOR, "div.content h1").text
            description = driver.find_element(By.CSS_SELECTOR, "div.content h2").text
        except:
            # Content not loaded properly
            item_time = time.time() - item_start
            print(f"❌ Content not loaded ({item_time:.1f}s)")
            all_scraped_data.append({
                "barcode": barcode,
                "part_number": "ERROR",
                "description": "Content failed to load (timeout or popup blocking)",
                "from_date": "",
                "to_date": "",
                "weight": "",
                "price": "",
                "vehicle_count": 0,
                "first_vehicle_tags": "",
                "scrape_time_seconds": round(item_time, 2)
            })
            continue
        
        # Extract details from dl
        part_details = {}
        try:
            dt_elements = driver.find_elements(By.CSS_SELECTOR, "div.content dl dt")
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "div.content dl dd")
            
            for dt, dd in zip(dt_elements, dd_elements):
                key = dt.text.replace(":", "").strip()
                value = dd.text.strip() if dd.text.strip() else "-"
                part_details[key] = value
        except:
            pass
        
        # Extract vehicle links
        vehicle_links_list = []
        try:
            links = driver.find_elements(By.CSS_SELECTOR, "div.partSearchResults ul li a")
            for link in links:
                vehicle_text = link.text
                vehicle_url = link.get_attribute("href")
                vehicle_links_list.append({"text": vehicle_text, "url": vehicle_url})
        except:
            pass
        
        # Step 4: Navigate to first vehicle link (if exists)
        vehicle_tags = ""
        if vehicle_links_list:
            first_vehicle_url = vehicle_links_list[0]["url"]
            safe_navigate(driver, first_vehicle_url)
            
            # Proactively kill popups before waiting
            time.sleep(0.5)  # Give page a moment to load
            aggressive_popup_killer(driver)
            
            # Try to wait for content, kill popups if timeout occurs
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".partSearchResults ul li"))
                )
            except:
                # If timeout, aggressively kill popups and try again
                aggressive_popup_killer(driver)
                time.sleep(0.5)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".partSearchResults ul li"))
                )
            
            # Immediately kill any popups that appeared
            aggressive_popup_killer(driver)
            
            # Step 5: Extract first vehicle tags
            try:
                results_section = driver.find_element(By.CLASS_NAME, "partSearchResults")
                first_li = results_section.find_element(By.CSS_SELECTOR, "ul li:first-child")
                full_text = first_li.text.strip()
                
                if ':' in full_text:
                    vehicle_tags = full_text.split(':')[0].strip()
                else:
                    vehicle_tags = full_text
            except:
                vehicle_tags = "N/A"
        
        # Calculate time taken
        item_time = time.time() - item_start
        
        # Store the collected data
        scraped_item = {
            "barcode": barcode,
            "part_number": part_number,
            "description": description,
            "from_date": part_details.get("From", ""),
            "to_date": part_details.get("To", ""),
            "weight": part_details.get("Weight", ""),
            "price": part_details.get("Price", ""),
            "vehicle_count": len(vehicle_links_list),
            "first_vehicle_tags": vehicle_tags,
            "scrape_time_seconds": round(item_time, 2)
        }
        
        all_scraped_data.append(scraped_item)
        
        print(f"✅ {part_number[:30]} | Vehicles: {len(vehicle_links_list)} ({item_time:.1f}s)")
        
    except Exception as e:
        # Calculate time even for errors
        item_time = time.time() - item_start
        
        # Get just the error type and message, not the full stack trace
        error_type = type(e).__name__
        error_msg = str(e).split('\n')[0][:200]  # First line, max 200 chars
        
        print(f"❌ {error_type}: {error_msg[:50]} ({item_time:.1f}s)")
        
        # Store error entry with clean error message
        all_scraped_data.append({
            "barcode": barcode,
            "part_number": "ERROR",
            "description": f"{error_type}: {error_msg}",
            "from_date": "",
            "to_date": "",
            "weight": "",
            "price": "",
            "vehicle_count": 0,
            "first_vehicle_tags": "",
            "scrape_time_seconds": round(item_time, 2)
        })
    
    # Minimal delay between requests
    time.sleep(0.3)

# Calculate total time and statistics
total_time = time.time() - start_time
end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
avg_time = total_time / len(barcodes) if barcodes else 0

print("\n" + "=" * 60)
print(f"✅ Scraping complete! Collected {len(all_scraped_data)} entries")
print(f"   Start: {start_datetime}")
print(f"   End: {end_datetime}")
print(f"   Total time: {total_time/60:.1f} minutes ({total_time:.1f} seconds)")
print(f"   Average per part: {avg_time:.1f} seconds")
print("=" * 60)

🚀 Starting optimized scraping for 79 barcodes...
   Start time: 2026-02-13 23:44:13

[1/79] [23:44:13] Processing: 34356790304 ✅ 34356790304 | Vehicles: 2 (22.3s)

[2/79] [23:44:35] Processing: 34116769951 ✅ 34116769951 | Vehicles: 1 (12.6s)

[3/79] [23:44:48] Processing: 34116860912 ✅ 34116860912 | Vehicles: 27 (16.5s)

[4/79] [23:45:05] Processing: 34216792228 ⚠️ NOT FOUND (7.3s)

[5/79] [23:45:12] Processing: 34116864057 ✅ 34116864057 | Vehicles: 2 (18.2s)

[6/79] [23:45:31] Processing: 34211166131 ✅ 34211166131 | Vehicles: 3 (13.7s)

[7/79] [23:45:45] Processing: 34216775291 ✅ 34216775291 | Vehicles: 8 (11.9s)

[8/79] [23:45:57] Processing: 34321159890 ✅ 34321159890 | Vehicles: 13 (12.1s)

[9/79] [23:46:10] Processing: 34356792564 ✅ 34356792564 | Vehicles: 13 (11.4s)

[10/79] [23:46:21] Processing: 34356778037 ✅ 34356778037 | Vehicles: 3 (12.0s)

[11/79] [23:46:33] Processing: 34356789330 ✅ 34356789330 | Vehicles: 8 (10.8s)

[12/79] [23:46:45] Processing: 34356778038 ✅ 34356778038 

KeyboardInterrupt: 

In [30]:
# ==============================
# PREVIEW SCRAPED DATA
# ==============================

print(f"Total entries: {len(all_scraped_data)}\n")

if all_scraped_data:
    print("First 5 entries:")
    print("=" * 100)
    
    for i, item in enumerate(all_scraped_data[:5], 1):
        print(f"\n{i}. Barcode: {item['barcode']}")
        print(f"   Part: {item['part_number']} - {item['description']}")
        print(f"   From: {item['from_date']} | To: {item['to_date']} | Weight: {item['weight']}")
        print(f"   Vehicles: {item['vehicle_count']} | Tags: {item['first_vehicle_tags']}")
    
    print("\n" + "=" * 100)
else:
    print("⚠️ No data scraped yet")

Total entries: 79

First 5 entries:

1. Barcode: 34356790304
   Part: 34356790304 - Brake pad wear sensor, rear
   From: 09/01/2010 | To: - | Weight: 0.057 kg
   Vehicles: 2 | Tags: X3 F25, X3 18d, SUV, N47N, EUR, (WY11)

2. Barcode: 34116769951
   Part: 34116769951 - Repair kit, brake pads asbestos-free
   From: 12/01/2004 | To: - | Weight: 1.626 kg
   Vehicles: 1 | Tags: 3' E90, 316i, Sedan, N45, EUR, (VA11)

3. Barcode: 34116860912
   Part: 34116860912 - Brake disc, lightweight,ventilated,right
   From: 07/01/2015 | To: 08/17/2023 (ENDED) | Weight: 12.656 kg
   Vehicles: 27 | Tags: 3' G20 Sedan, 316d, Sedan, B47B, EUR, (5W70)

4. Barcode: 34216792228
   Part: ERROR - Message: 
Stacktrace:
0   undetected_chromedriver             0x00000001030943f8 undetected_chromedriver + 6730744
1   undetected_chromedriver             0x000000010308bc2a undetected_chromedriver + 6695978
2   undetected_chromedriver             0x0000000102a97cf5 undetected_chromedriver + 453877
3   undetected_chrome

In [ ]:
# ==============================
# SAVE TO CSV FILE
# ==============================

output_file = "scraped_bmw_parts.csv"

try:
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        if all_scraped_data:
            # Get all unique keys from all entries
            fieldnames = list(all_scraped_data[0].keys())
            
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_scraped_data)
            
            print(f"✅ Data saved to: {output_file}")
            print(f"   Total rows: {len(all_scraped_data)}")
            print(f"   Columns: {', '.join(fieldnames)}")
        else:
            print("⚠️ No data to save")
            
except Exception as e:
    print(f"❌ Error saving file: {e}")

In [31]:
# ==============================
# SAVE TO EXCEL FILE
# ==============================

from openpyxl import Workbook

output_excel = "scraped_bmw_parts.xlsx"

try:
    wb_output = Workbook()
    ws = wb_output.active
    ws.title = "BMW Parts Data"
    
    if all_scraped_data:
        # Write headers
        headers = list(all_scraped_data[0].keys())
        ws.append(headers)
        
        # Write data rows
        for item in all_scraped_data:
            row = [item.get(h, "") for h in headers]
            ws.append(row)
        
        # Auto-adjust column widths
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 50)
            ws.column_dimensions[column_letter].width = adjusted_width
        
        wb_output.save(output_excel)
        
        print(f"✅ Data saved to: {output_excel}")
        print(f"   Total rows: {len(all_scraped_data)}")
        print(f"   Columns: {', '.join(headers)}")
    else:
        print("⚠️ No data to save")
        
except Exception as e:
    print(f"❌ Error saving Excel file: {e}")

✅ Data saved to: scraped_bmw_parts.xlsx
   Total rows: 79
   Columns: barcode, part_number, description, from_date, to_date, weight, price, vehicle_count, first_vehicle_tags


In [ ]:
# ==============================
# CLEANUP - CLOSE DRIVER
# ==============================

try:
    driver.quit()
    print("✅ Browser closed successfully")
except Exception as e:
    print(f"⚠️ Error closing browser: {e}")